In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os
import numpy as np

# 데이터셋 로드

In [3]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


# 분산 전력 정의
- 분산 관련 처리를 하는 ```MirroredStrategy``` 객체를 생성

In [4]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [5]:
print("장치의 수: {}".format(strategy.num_replicas_in_sync))

장치의 수: 1


# 입력 파이프라인 구성

In [6]:
# 샘플의 수는 info.splits.total_num_examples로도 얻을 수 있음

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

# 정규화

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

In [8]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

# 모델 정의

In [9]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


# 콜백 정의
콜백은 다음과 같다.
- 텐서보드(TensorBoard) : 텐서보드용 로그를 남겨, 그래프를 그릴 수 있게 해줌.
- 체크포인트(checkpoint) : 매 epoch이 끝난 후 모델을 저장.
- 스케쥴러(scheduler) : 매 epoch 또는 batch마다 learning_rate를 조절할 수 있다.

In [10]:
# 체크포인트 

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [11]:
# 스케쥴러

def decay(epoch):
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

In [12]:
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\npoech {}의 learning_rate는 {}입니다.'.format(epoch + 1, model.optimizer.lr.numpy()))

In [13]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

# Training & Evaluation

In [14]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


938/938 [==============================] - ETA: 0s - loss: 0.2068 - accuracy: 0.9396
poech 1의 learning_rate는 0.0010000000474974513입니다.
938/938 [==============================] - 23s 10ms/step - loss: 0.2068 - accuracy: 0.9396 - lr: 0.0010
Epoch 2/12
926/938 [============================>.] - ETA: 0s - loss: 0.0678 - accuracy: 0.9795
poech 2의 learning_rate는 0.0010000000474974513입니다.
938/938 [==============================] - 7s 7ms/step - loss: 0.0675 - accuracy: 0.9796 - lr: 0.0010
Epoch 3/12
938/938 [==============================] - ETA: 0s - loss: 0.0460 - accuracy: 0.9864
poech 3의 learning_rate는 0.0010000000474974513입니다.
938/938 [==============================] - 7s 7ms/step - loss: 0.0460 - accuracy: 0.9864 - lr: 0.0010
Epoch 4/12
927/938 [============================>.] - ETA: 0s - loss: 0.0257 - accuracy: 0.9932
poech 4의 learning_rate는 9.999999747378752e-05입니다.
938/938 [==============================] - 6s 6ms/step - loss: 0.0257 - accuracy: 0.9932 - lr: 1.0000e-04
Epoch 5/12
93

In [15]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print("loss: {}, acc: {}".format(eval_loss, eval_acc))

157/157 [==============================] - 3s 9ms/step - loss: 0.0401 - accuracy: 0.9869
loss: 0.040143270045518875, acc: 0.9868999719619751


# 모델 저장 및 불러오기

In [16]:
# 모델 전체 저장 및 로드

model.save('trained_model')
new_model = tf.keras.models.load_model('trained_model')

# 새로운 모델 평가
eval_loss, eval_acc = new_model.evaluate(eval_dataset)
print("loss: {}, acc: {}".format(eval_loss, eval_acc))

INFO:tensorflow:Assets written to: trained_model/assets


INFO:tensorflow:Assets written to: trained_model/assets


157/157 [==============================] - 1s 3ms/step - loss: 0.0401 - accuracy: 0.9869
loss: 0.040143270045518875, acc: 0.9868999719619751


In [18]:
# 모델 가중치 저장 및 로드

model.save_weights('model_weight')

new_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

new_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

new_model.load_weights('model_weight')


# 새로운 모델 평가
eval_loss, eval_acc = new_model.evaluate(eval_dataset)
print("loss: {}, acc: {}".format(eval_loss, eval_acc))

157/157 [==============================] - 1s 4ms/step - loss: 0.0401 - accuracy: 0.9869
loss: 0.040143270045518875, acc: 0.9868999719619751


### 모델의 성능이 동일한 것을 알 수 있다.